In [1]:
import yt
import glob
import numpy as np
import matplotlib.pyplot as plt
import argparse
from mpl_toolkits.axes_grid1 import AxesGrid
from satellite_analysis.catalogreaders import consistentcatalogreader as consistent
from satellite_analysis.catalogreaders import tomercatalogreader as tomer
from satellite_analysis.graphs import stellarmassrelationfindingrvirprojections as rvirprj


#need to add arguments for VELA_dir and input_dir for the rockstarcatalogreader

def parse():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('input_dir')
    parser.add_argument('VELA_dir')
    parser.add_argument('--subhalos', nargs='?', default=True)
    args = vars(parser.parse_args())
    return args

#args = parse()
#print(args)

#input_dir = args['input_dir']
#VELA_dir = args['VELA_dir']
#subhalos = args['subhalos']

input_dir = '/Users/user1/documents/VELA07/hlists'
VELA_dir = '/Users/user1/documents/VELA07'
VELA_number = '07'
subhalos = False

#Now run the rockstarcatalogreader to get the halo location data.

#input_dir = '/Users/user1/Documents/VELA07/baserockstar_ascii/subhalos/'
consistent.consistent_catalog_reader(input_dir, subhalos=subhalos)

#Collect the names of the VELA simulations in the VELA_dir so that star data can be extracted.
VELA_snaps = glob.glob(VELA_dir + '/10MpcBox*')
VELA_snaps.sort()

#Extract the scale factors from the VELA snapshot files to match to the rockstar catalogs.
VELA_index = []
for snap in VELA_snaps:
    period = [pos for pos, char in enumerate(snap) if char == '.']
    number = snap[period[-2]+1:period[-1]]
    VELA_index.append(number)
VELA_index.sort()

#load the tomer catalogs
tomer.read_tomer(VELA_number)
tomer_scales = [str(scale)[2:5] for scale in tomer.tomer_list['aexpn'].tolist()]
r_vir_tomer = tomer.tomer_list['r_vir[kpc]'].tolist()
x_tomer = tomer.tomer_list['center[0](code)'].tolist()
y_tomer = tomer.tomer_list['center[1](code)'].tolist()
z_tomer = tomer.tomer_list['center[2](code)'].tolist()

#Loop over the rockstar catalogs for each snapshot.
for index in consistent.snapshot_index:
    VELA_a = consistent.consistent_file_index[index]
    position = [pos for pos, loc in enumerate(VELA_index) if loc == VELA_a]
    if position == [] or len(position) > 1:
        print('Could not find corresponding VELA 10Mpc File for snapshot:', VELA_a)
    else:
        print('Finding MVir Masses for snap:', VELA_a)
        #load the yt snap and the halo_data from the catalog reader
        ds = yt.load(VELA_snaps[position[0]])
        halo_data = consistent.halo_data_largest[index][0]
        
        
        #define the parameters for the largest halo for plotting
        domain_width = float(ds.domain_width.in_units('Mpc/h')[0])
        largest_halo = halo_data[0]
        x = float(largest_halo[17])/domain_width
        y = float(largest_halo[18])/domain_width
        z = float(largest_halo[19])/domain_width
        rvir = float(largest_halo[11])
        center = [x, y, z]
        
        tomer_number = [pos for pos, number in enumerate(tomer_scales) if number == VELA_a]
        if tomer_number == [] or len(tomer_number) > 1:
            print('Could not find corresponding Tomer Snapshot Data for snap', VELA_a)
            tomer_center = [0,0,0]
            tomer_rvir = [0]
        else:
            tomer_center = [x_tomer[tomer_number[0]], y_tomer[tomer_number[0]], z_tomer[tomer_number[0]]]
            tomer_rvir = r_vir_tomer[tomer_number[0]]
         
        
        rvirprj.plotting_xyz_projection(ds, 10, center, rvir, tomer_center, tomer_rvir, VELA_a, input_dir)
        #just like above, add the circles before the plots are added to the grid

/Users/user1/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Consistent File Indices: ['100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '210', '220', '230', '240', '250', '260', '270', '280', '290', '300', '310', '320', '330', '340', '350', '360', '370', '380', '390', '400', '410', '420', '430', '440', '450', '460', '470', '480', '490', '500', '510', '520', '530', '540']

Using Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]

Number of Halos found for snapshot 0 : 3058
Number of Halos found for snapshot 1 : 3437
Number of Halos found for snapshot 2 : 3137
Number of Halos found for snapshot 3 : 2711
Number of Halos found for snapshot 4 : 2421
Number of Halos found for snapshot 5 : 1877
Number of Halos found for snapshot 6 : 1234
Number of Halos found for snapshot 7 : 1880
Number of Halos found for snapshot 8 : 1861
Number of Halos found for snapshot 9 : 1912
Number of Halos found for snapshot 10 : 22

/Users/user1/anaconda3/lib/python3.6/site-packages/satellite_analysis/catalogreaders/tomercatalogreader.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cen_gal_cat_s = cen_gal_cat[cen_gal_cat['gen'] == 'VELA_v2'][cen_gal_cat['galaxynumber'] == galaxynumber][cen_gal_cat['type'] == 'Thick']


Could not find corresponding VELA 10Mpc File for snapshot: 100
Could not find corresponding VELA 10Mpc File for snapshot: 110
Could not find corresponding VELA 10Mpc File for snapshot: 120
Could not find corresponding VELA 10Mpc File for snapshot: 130
Could not find corresponding VELA 10Mpc File for snapshot: 140
Could not find corresponding VELA 10Mpc File for snapshot: 150
Could not find corresponding VELA 10Mpc File for snapshot: 160
Could not find corresponding VELA 10Mpc File for snapshot: 170
Could not find corresponding VELA 10Mpc File for snapshot: 180
Could not find corresponding VELA 10Mpc File for snapshot: 190
Could not find corresponding VELA 10Mpc File for snapshot: 210
Could not find corresponding VELA 10Mpc File for snapshot: 220
Could not find corresponding VELA 10Mpc File for snapshot: 230
Could not find corresponding VELA 10Mpc File for snapshot: 240
Finding MVir Masses for snap: 250


yt : [INFO     ] 2019-05-13 19:31:35,554 discovered particle_header:/Users/user1/documents/VELA07/PMcrda0.250.DAT
yt : [INFO     ] 2019-05-13 19:31:35,555 discovered particle_data:/Users/user1/documents/VELA07/PMcrs0a0.250.DAT
yt : [INFO     ] 2019-05-13 19:31:35,557 discovered particle_stars:/Users/user1/documents/VELA07/stars_a0.250.dat
yt : [INFO     ] 2019-05-13 19:31:35,571 Using root level of 14
yt : [INFO     ] 2019-05-13 19:31:35,573 Discovered 7 species of particles
yt : [INFO     ] 2019-05-13 19:31:35,575 Particle populations:  34930688   4943872    855040    139392     21776   2088309   1735376 
yt : [INFO     ] 2019-05-13 19:31:35,629 Max level is 12
yt : [INFO     ] 2019-05-13 19:31:35,645 Parameters: current_time              = 2.2311757832017673 Gyr
yt : [INFO     ] 2019-05-13 19:31:35,646 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2019-05-13 19:31:35,647 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-05-13 19:31:35,6

...

yt : [INFO     ] 2019-05-13 19:40:30,323 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:30,324 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:30,326 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:30,328 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:30,330 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:40:35,260 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:35,261 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:35,263 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:35,265 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:40:35,267 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:40:40,142 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:40:40,279 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:40:40

Could not find corresponding VELA 10Mpc File for snapshot: 410
Could not find corresponding VELA 10Mpc File for snapshot: 420
Could not find corresponding VELA 10Mpc File for snapshot: 430
Could not find corresponding VELA 10Mpc File for snapshot: 440
Could not find corresponding VELA 10Mpc File for snapshot: 450
Could not find corresponding VELA 10Mpc File for snapshot: 460
Could not find corresponding VELA 10Mpc File for snapshot: 470
Could not find corresponding VELA 10Mpc File for snapshot: 480
Could not find corresponding VELA 10Mpc File for snapshot: 490
Finding MVir Masses for snap: 500


yt : [INFO     ] 2019-05-13 19:41:00,569 discovered particle_header:/Users/user1/documents/VELA07/PMcrda0.500.DAT
yt : [INFO     ] 2019-05-13 19:41:00,571 discovered particle_data:/Users/user1/documents/VELA07/PMcrs0a0.500.DAT
yt : [INFO     ] 2019-05-13 19:41:00,574 discovered particle_stars:/Users/user1/documents/VELA07/stars_a0.500.dat
yt : [INFO     ] 2019-05-13 19:41:00,597 Using root level of 14
yt : [INFO     ] 2019-05-13 19:41:00,599 Discovered 7 species of particles
yt : [INFO     ] 2019-05-13 19:41:00,603 Particle populations:  34930688   4943872    855040    139392     21776   2088309   6259527 
yt : [INFO     ] 2019-05-13 19:41:00,658 Max level is 12
yt : [INFO     ] 2019-05-13 19:41:00,669 Parameters: current_time              = 6.035152709948895 Gyr
yt : [INFO     ] 2019-05-13 19:41:00,670 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2019-05-13 19:41:00,672 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2019-05-13 19:41:00,67

yt : [INFO     ] 2019-05-13 19:43:55,050 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:43:55,051 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:43:55,053 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:43:55,055 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:43:55,057 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:44:02,391 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:44:02,393 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:44:02,396 xlim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:44:02,399 ylim = 0.000000 1.000000
yt : [INFO     ] 2019-05-13 19:44:02,403 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:44:12,224 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:44:12,421 Splatting (('darkmatter4', 'particle_mass')) onto a 800 by 800 mesh
yt : [INFO     ] 2019-05-13 19:44:12

Could not find corresponding VELA 10Mpc File for snapshot: 510
Could not find corresponding VELA 10Mpc File for snapshot: 520
Could not find corresponding VELA 10Mpc File for snapshot: 530
Could not find corresponding VELA 10Mpc File for snapshot: 540


In [24]:
haloid1, haloid2, haloid3, haloid4 = [], [], [], []
for halo in consistent.halo_data_largest[14][0]:
    haloid1.append(halo[6])
for halo in consistent.halo_data_largest[22][0]:
    haloid2.append(halo[6])
for halo in consistent.halo_data_largest[29][0]:
    haloid3.append(halo[6])
for halo in consistent.halo_data_largest[39][0]:
    haloid4.append(halo[6])


print(consistent.consistent_index)
print('')
print(consistent.halo_data_largest[14][0][0])
print('')
print(haloid1)
print('')
print(consistent.halo_data_largest[22][0][0])
print('')
print(haloid2)
print('')
print(consistent.halo_data_largest[29][0][0])
print('')
print(haloid3)
print(consistent.halo_data_largest[29][0][2])
print('')
print(consistent.halo_data_largest[39][0][0])
print('')
print(haloid4)
print('')

#scale(0) id(1) desc_scale(2) desc_id(3) num_prog(4) pid(5) upid(6) desc_pid(7) phantom(8) sam_Mvir(9) Mvir(10) Rvir(11) rs(12) vrms(13) mmp?(14) scale_of_last_MM(15) vmax(16) x(17) y(18) z(19) vx(20) vy(21) vz(22) Jx(23) Jy(24) Jz(25) Spin(26) Breadth_first_ID(27) Depth_first_ID(28) Tree_root_ID(29) Orig_halo_ID(30) Snap_idx(31) Next_coprogenitor_depthfirst_ID(32) Last_progenitor_depthfirst_ID(33) Last_mainleaf_depthfirst_ID(34) Tidal_Force(35) Tidal_ID(36) Rs_Klypin(37) Mvir_all(38) M200b(39) M200c(40) M500c(41) M2500c(42) Xoff(43) Voff(44) Spin_Bullock(45) b_to_a(46) c_to_a(47) A[x](48) A[y](49) A[z](50) b_to_a(500c)(51) c_to_a(500c)(52) A[x](500c)(53) A[y](500c)(54) A[z](500c)(55) T/|U|(56) M_pe_Behroozi(57) M_pe_Diemer(58) Halfmass_Radius(59) Macc(60) Mpeak(61) Vacc(62) Vpeak(63) Halfmass_Scale(64) Acc_Rate_Inst(65) Acc_Rate_100Myr(66) Acc_Rate_1*Tdyn(67) Acc_Rate_2*Tdyn(68) Acc_Rate_Mpeak(69) Acc_Log_Vmax_Inst(70) Acc_Log_Vmax_1*Tdyn(71) Mpeak_Scale(72) Acc_Scale(73) First_Acc_Sc

In [4]:
list500, list400 = [], []
listid3144750 = []
listid3572034 = []
for lists in consistent.halo_data_all:
    for halo in lists:
        if halo[7] == '3572034':
            list500.append(halo)
        if halo[7] == '3144750':
            list400.append(halo)
        if halo[1] == '3144750':
            listid3144750.append(halo)
        if halo[1] == '3572034':
            listid3572034.append(halo)
            

IndexError: list index out of range

In [21]:
print(listid3144750)
print('')
print(listid3572034)

[['0.40009', '3144750', '0.41023', '3197797', '1941', '-1', '-1', '-1', '0', '1.71256e+12', '1.30800e+12', '278.337000', '37.610000', '238.010000', '1', '0.34050', '291.790000', '7.99123', '7.61825', '8.28907', '20.550', '8.710', '-72.690', '-2.722e+12', '-7.098e+10', '-1.127e+12', '0.07562', '82572', '14', '3691463', '2603', '29', '405287', '405286', '43', '0.00000', '-1', '20.3496', '1307700000000', '1300900000000', '1268300000000', '1099100000000', '733490000000', '57.3378', '98.63', '0.06366', '0.20403', '0.16516', '-2.4245', '6.19117', '9.82029', '0.10194', '0.07574', '-2.87256', '7.19964', '11.6987', '0.4403', '1506000000000', '1005000000000', '83.352', '1.30800e+12', '1.31800e+12', '291.790000', '349.220000', '0.32047', '-6.450e+00', '-6.450e+00', '2.960e+01', '7.822e+02', '5.650e+02', '-1.945e-10', '2.977e-11', '0.38021', '0.40009', '0.40009', '1.308e+12', '291.790', '336.750', '0.00000', '0.02060', '-1.00000', '3144750']]

[['0.50060', '3572034', '0.51036', '3604137', '897', '

In [6]:
for halo in consistent.halo_data_largest[40][0]:
    if halo[1] == '3572034':
        print(halo)